# Chapter 5
## Structured APIs - Basic Structured Operations

### Imports

In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import col, column, expr, lit, desc, asc

### Instantiate a Spark Session

In [2]:
spark = SparkSession.builder.appName("Chapter5").getOrCreate()

### Files

In [3]:
flight_data_2015 = "./Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json"
all_flight_data = "./Spark-The-Definitive-Guide/data/flight-data/json/*-summary.json"

### Load files into Dataframes

In [4]:
df = spark.read.format("json").load(flight_data_2015)

In [5]:
spark.read.format("json").load(flight_data).schema

NameError: name 'flight_data' is not defined

In [ ]:
myManualSchema = StructType([
  StructField("DEST_COUNTRY_NAME", StringType(), True),
  StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
  StructField("count", LongType(), False, metadata={"hello":"world"})
])

df = spark.read.format("json").schema(myManualSchema)\
  .load(flight_data)

df.schema

In [ ]:
col("someColumnName")
column("someColumnName")

In [ ]:
expr("(((someCol + 5) * 200) - 6) < otherCol")

In [ ]:
myRow = Row("Hello", None, 1, False)


In [ ]:
myRow[0]

In [ ]:
myRow[2]

In [ ]:
df.createOrReplaceTempView("dfTable")

In [ ]:
myManualSchema = StructType([
  StructField("some", StringType(), True),
  StructField("col", StringType(), True),
  StructField("names", LongType(), False)
])


In [ ]:
myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

In [ ]:
df.select("DEST_COUNTRY_NAME").show(2)

In [ ]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

In [ ]:
df.select(
    expr("DEST_COUNTRY_NAME"),
    col("DEST_COUNTRY_NAME"),
    column("DEST_COUNTRY_NAME"))\
  .show(2)

In [ ]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

In [ ]:
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME"))\
  .show(2)

In [ ]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

In [ ]:
df.selectExpr(
  "*", # all original columns
  "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
  .show(2)

In [ ]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

In [ ]:
df.select(expr("*"), lit(1).alias("One")).show(2)

In [ ]:
df.withColumn("numberOne", lit(1)).show(2)

In [ ]:
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME"))\
  .show(2)

In [ ]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns


In [ ]:
dfWithLongColName = df.withColumn(
    "This Long Column-Name",
    expr("ORIGIN_COUNTRY_NAME"))

In [ ]:
dfWithLongColName.selectExpr(
    "`This Long Column-Name`",
    "`This Long Column-Name` as `new col`")\
  .show(2)

In [ ]:
dfWithLongColName.select(expr("`This Long Column-Name`")).columns

In [ ]:
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia")\
  .show(2)

In [ ]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

In [ ]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

In [ ]:
seed = 5
withReplacement = False
fraction = 0.5

df.sample(withReplacement, fraction, seed).count()

In [ ]:
dataFrames = df.randomSplit([0.25, 0.75], seed)

In [ ]:
dataFrames[0].count() > dataFrames[1].count()

In [ ]:
schema = df.schema
newRows = [
  Row("New Country", "Other Country", 5),
  Row("New Country 2", "Other Country 3", 1)
]
parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(parallelizedRows, schema)

In [ ]:
df.union(newDF)\
  .where("count = 1")\
  .where(col("ORIGIN_COUNTRY_NAME") != "United States")\
  .show()

In [ ]:
df.sort("count").show(5)

In [ ]:
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)

In [ ]:
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

In [ ]:
df.orderBy(expr("count desc")).show(2)

In [ ]:
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

In [ ]:
spark.read.format("json").load(all_flight_data)\
  .sortWithinPartitions("count")

In [ ]:
df.limit(5).show()

In [ ]:
df.orderBy(expr("count desc")).limit(6).show()

In [ ]:
df.rdd.getNumPartitions() # 1

In [ ]:
df.repartition(5)

In [ ]:
df.repartition(col("DEST_COUNTRY_NAME"))

In [ ]:
df.repartition(5, col("DEST_COUNTRY_NAME"))

In [ ]:
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

In [ ]:
collectDF = df.limit(10)
collectDF.take(5) # take works with an Integer count
collectDF.show() # this prints it out nicely
collectDF.show(5, False)
collectDF.collect()